# 03. Build Master Dataset (Mean Version)

## 개요
본 노트북은 SECOM 데이터의 **전처리 마스터셋(Stage A)** 을 생성합니다.  
- **목적**: 결측치 처리, 다중공선성 제거, 이상치 완화를 거쳐 모델링 준비 상태의 베이스라인 데이터셋 구축
- **범위**: Stage A만 수행 (스케일링/샘플링/Feature Selection은 CV 내부에서 수행 예정)
- **버전**: Median imputation only

## 출력물
- `data/processed/base_master_mean.parquet` : 최종 마스터셋
- `data/processed/preprocess_log_mean.json` : 전처리 로그
- `data/interim/preview_master_mean.csv` : 상위 200행 미리보기

### 환경 세팅

In [1]:
# 환경 세팅 & 라이브러리/경로 선언
import pyarrow as pa
import pandas as pd
import numpy as np
import json
from pathlib import Path
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 버전 확인
print("pandas:", pd.__version__, "pyarrow:", pa.__version__)

# 프로젝트 루트 자동 설정
ROOT = Path.cwd().parent  # notebooks 폴더의 상위

# ✅ 이미 timestamp 제거된 버전 사용
DATA_RAW = ROOT / 'data/secom_model_train.csv'

# 디렉토리 경로
DIR_INTERIM = ROOT / 'data/interim'
DIR_PROC = ROOT / 'data/processed'

# 디렉토리 생성 보장
DIR_INTERIM.mkdir(parents=True, exist_ok=True)
DIR_PROC.mkdir(parents=True, exist_ok=True)

# 재현성 설정
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ 환경 세팅 완료")
print(f"  - 원본 데이터: {DATA_RAW}")
print(f"  - 출력 디렉토리: {DIR_PROC}")

pandas: 2.3.3 pyarrow: 21.0.0
✓ 환경 세팅 완료
  - 원본 데이터: /Users/mealkuo/Desktop/capstone02_project/data/secom_model_train.csv
  - 출력 디렉토리: /Users/mealkuo/Desktop/capstone02_project/data/processed


## 데이터 로드 & 라벨 매핑

**라벨 인코딩 규칙**:
- `-1` (Pass) → `0` (정상)
- `1` (Fail) → `1` (불량)
- Positive class = 1 (불량)

기본 통계 및 결측률을 확인합니다.

In [2]:
# 데이터 로드 & 라벨 매핑
df = pd.read_csv(DATA_RAW)
print(f"원본 데이터 shape: {df.shape}")

# 라벨 매핑: 1 -> 1 (불량), -1 -> 0 (정상)
label_map = {-1: 0, 1: 1}
df['label'] = df['label'].map(label_map)

# y와 X 분리
y = df['label'].copy()
X = df.drop(columns=['label'])

print(f"\n✓ 라벨 매핑 완료")
print(f"  - X shape: {X.shape}")
print(f"  - Positive class ratio: {y.mean():.4f}")
print(f"  - Class distribution:\n{y.value_counts().sort_index()}")

# 초기 결측률 확인
missing_ratios = X.isnull().mean().sort_values(ascending=False)
print(f"\n결측률 상위 10개 컬럼:")
print(missing_ratios.head(10))

원본 데이터 shape: (1567, 591)

✓ 라벨 매핑 완료
  - X shape: (1567, 590)
  - Positive class ratio: 0.0664
  - Class distribution:
label
0    1463
1     104
Name: count, dtype: int64

결측률 상위 10개 컬럼:
sensor_158    0.911934
sensor_293    0.911934
sensor_294    0.911934
sensor_159    0.911934
sensor_493    0.855775
sensor_359    0.855775
sensor_086    0.855775
sensor_221    0.855775
sensor_247    0.649649
sensor_110    0.649649
dtype: float64


## 결측률 기준 컬럼 제거

**기준**: 결측률 ≥ 0.40인 컬럼을 제거합니다.  
제거된 컬럼 목록은 로그에 기록됩니다.

In [3]:
# 결측률 기반 컬럼 제거
MISSING_THRESHOLD = 0.40

missing_ratios = X.isnull().mean()
cols_to_drop_missing = missing_ratios[missing_ratios >= MISSING_THRESHOLD].index.tolist()

print(f"결측률 ≥ {MISSING_THRESHOLD} 컬럼 수: {len(cols_to_drop_missing)}")
if len(cols_to_drop_missing) > 0:
    print(f"제거 대상 컬럼 예시 (최대 5개): {cols_to_drop_missing[:5]}")

X = X.drop(columns=cols_to_drop_missing)
print(f"\n✓ 결측률 기반 제거 완료")
print(f"  - 남은 컬럼 수: {X.shape[1]}")
print(f"  - 제거된 컬럼 수: {len(cols_to_drop_missing)}")

결측률 ≥ 0.4 컬럼 수: 32
제거 대상 컬럼 예시 (최대 5개): ['sensor_073', 'sensor_074', 'sensor_086', 'sensor_110', 'sensor_111']

✓ 결측률 기반 제거 완료
  - 남은 컬럼 수: 558
  - 제거된 컬럼 수: 32


## 결측치 Mean 대치

남은 모든 NaN 값을 각 피처별 **mean** 값으로 대치합니다.

In [4]:
# 결측치 mean 대치
nan_count_before = X.isnull().sum().sum()
print(f"대치 전 NaN 총 개수: {nan_count_before}")

for col in X.columns:
    if X[col].isnull().any():
        mean_val = X[col].mean()
        X[col].fillna(mean_val, inplace=True)

nan_count_after = X.isnull().sum().sum()
print(f"\n✓ Mean 대치 완료")
print(f"  - 대치 후 NaN 총 개수: {nan_count_after}")
assert nan_count_after == 0, "NaN이 남아있습니다!"

대치 전 NaN 총 개수: 8823

✓ Mean 대치 완료
  - 대치 후 NaN 총 개수: 0


/var/folders/5j/xncx4f310w93nzyd92xch9jh0000gn/T/ipykernel_37337/3784397956.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(mean_val, inplace=True)


## Z-score 이상치 완화

**전략**: 각 피처에서 |z-score| > 3인 값을 해당 피처의 median으로 대치합니다.
- 목적: 극단값 완화 (제거가 아닌 완화)
- 표준편차가 0인 컬럼은 스킵

=> mean은 이상치에 민감하기 때문에, 동일하게 median으로 처리

In [5]:
# Z-score 이상치 완화
Z_THRESHOLD = 3

outlier_replaced_counts = {}

for col in X.columns:
    col_std = X[col].std()
    
    # 표준편차가 0이면 스킵
    if col_std == 0:
        continue
    
    z_scores = np.abs(stats.zscore(X[col]))
    outliers_mask = z_scores > Z_THRESHOLD
    outliers_count = outliers_mask.sum()
    
    if outliers_count > 0:
        median_val = X[col].median()
        X.loc[outliers_mask, col] = median_val
        outlier_replaced_counts[col] = int(outliers_count)

total_replaced = sum(outlier_replaced_counts.values())
print(f"✓ Z-score 이상치 완화 완료")
print(f"  - 대치된 총 값 개수: {total_replaced}")
print(f"  - 영향받은 컬럼 수: {len(outlier_replaced_counts)}")
if len(outlier_replaced_counts) > 0:
    top_5 = sorted(outlier_replaced_counts.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"  - 상위 5개 컬럼: {top_5}")

✓ Z-score 이상치 완화 완료
  - 대치된 총 값 개수: 6065
  - 영향받은 컬럼 수: 409
  - 상위 5개 컬럼: [('sensor_039', 71), ('sensor_577', 70), ('sensor_575', 68), ('sensor_578', 62), ('sensor_573', 60)]


## 저분산/상수 컬럼 제거

**제거 기준**:
1. `VarianceThreshold(0.0)`: 분산이 0인 컬럼
2. 유니크 값이 1개인 컬럼 (안전망)

두 조건을 모두 체크하여 상수 컬럼을 제거합니다.

In [6]:
# 저분산/상수 컬럼 제거
selector = VarianceThreshold(threshold=0.0)
selector.fit(X)
cols_variance = X.columns[selector.get_support()].tolist()

# 안전망: 유니크값이 1개인 컬럼 추가 체크
cols_unique = [col for col in X.columns if X[col].nunique() > 1]

# 교집합으로 유지할 컬럼 결정
cols_to_keep = list(set(cols_variance) & set(cols_unique))
cols_to_drop_lowvar = list(set(X.columns) - set(cols_to_keep))

print(f"저분산/상수 컬럼 수: {len(cols_to_drop_lowvar)}")
if len(cols_to_drop_lowvar) > 0:
    print(f"제거 대상 컬럼 예시 (최대 5개): {cols_to_drop_lowvar[:5]}")

X = X[cols_to_keep]
print(f"\n✓ 저분산/상수 컬럼 제거 완료")
print(f"  - 남은 컬럼 수: {X.shape[1]}")
print(f"  - 제거된 컬럼 수: {len(cols_to_drop_lowvar)}")

저분산/상수 컬럼 수: 116
제거 대상 컬럼 예시 (최대 5개): ['sensor_539', 'sensor_263', 'sensor_330', 'sensor_243', 'sensor_326']

✓ 저분산/상수 컬럼 제거 완료
  - 남은 컬럼 수: 442
  - 제거된 컬럼 수: 116


## 상관관계 기반 피처 필터링 (Pre-VIF)

- **목적:**  
  다중공선성(VIF) 계산 전에, 서로 **너무 비슷하게 움직이는 센서들(상관계수 ≥ 0.95)** 을 먼저 제거하여  
  연산 속도를 개선하고, 불필요한 중복 신호를 정리함.

- **로직 요약:**  
  1️⃣ 센서 간 상관계수 행렬 계산 (`X.corr().abs()`)  
  2️⃣ 상삼각행렬만 남겨 중복 제거  
  3️⃣ |r| ≥ 0.95 이상인 컬럼 중 하나를 Drop  
  4️⃣ 남은 피처들만 다음 단계(VIF 기반 제거)에 전달

- **이유:**  
  SECOM 센서 데이터는 같은 라인에서 측정된 연속 피처가 많기 때문에,  
  상관계수 기반 선제적 필터링을 하지 않으면 **VIF 값이 폭발하거나 수렴 속도가 매우 느려질 수 있음.**

In [7]:
# =============================================
#  상관관계 기반 피처 제거 (VIF 이전 단계)
# =============================================

print(f"상관 필터링 시작 (초기 컬럼 수: {X.shape[1]})")

# 절댓값 상관계수 행렬 계산
corr_matrix = X.corr().abs()

# 상삼각행렬만 남기기 (A-B와 B-A 중복 제거)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.97 이상인 컬럼 추출
drop_corr = [column for column in upper.columns if any(upper[column] > 0.97)]

# 피처 제거
X = X.drop(columns=drop_corr)
print(f"✓ 1차 상관 필터링 완료")
print(f"  - 제거된 컬럼 수: {len(drop_corr)}")
if len(drop_corr) > 0:
    print(f"  - 제거된 컬럼 예시 (최대 5개): {drop_corr[:5]}")
print(f"  - 남은 컬럼 수: {X.shape[1]}")

상관 필터링 시작 (초기 컬럼 수: 442)
✓ 1차 상관 필터링 완료
  - 제거된 컬럼 수: 92
  - 제거된 컬럼 예시 (최대 5개): ['sensor_438', 'sensor_458', 'sensor_442', 'sensor_199', 'sensor_569']
  - 남은 컬럼 수: 350


## VIF 기반 다중공선성 제거

**전략**: VIF > 10인 컬럼을 반복적으로 제거합니다.
- 매 반복마다 가장 높은 VIF를 가진 컬럼 1개를 제거
- 모든 VIF ≤ 10이 되거나 최대 반복 횟수에 도달하면 중단
- 최대 반복: 100회

In [8]:
# VIF 기반 다중공선성 제거
VIF_THRESHOLD = 10
MAX_ITER = 100

X_vif = X.copy()
dropped_vif_cols = []

print(f"VIF 제거 시작 (초기 컬럼 수: {X_vif.shape[1]})")

for iteration in range(MAX_ITER):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_vif.columns
    vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    
    max_vif = vif_data['VIF'].max()
    
    if max_vif <= VIF_THRESHOLD:
        print(f"  반복 {iteration}: 모든 VIF ≤ {VIF_THRESHOLD} 달성")
        break
    
    # 가장 높은 VIF 컬럼 제거
    col_to_drop = vif_data.loc[vif_data['VIF'].idxmax(), 'Feature']
    dropped_vif_cols.append(col_to_drop)
    X_vif = X_vif.drop(columns=[col_to_drop])
    
    if (iteration + 1) % 10 == 0:
        print(f"  반복 {iteration + 1}: Max VIF = {max_vif:.2f}, 제거된 컬럼 수 = {len(dropped_vif_cols)}")

X = X_vif
print(f"\n✓ VIF 제거 완료")
print(f"  - 남은 컬럼 수: {X.shape[1]}")
print(f"  - 제거된 컬럼 수: {len(dropped_vif_cols)}")
if len(dropped_vif_cols) > 0:
    print(f"  - 제거된 컬럼 예시 (최대 5개): {dropped_vif_cols[:5]}")

VIF 제거 시작 (초기 컬럼 수: 350)
  반복 10: Max VIF = 178909.22, 제거된 컬럼 수 = 10
  반복 20: Max VIF = 58302.19, 제거된 컬럼 수 = 20
  반복 30: Max VIF = 15719.95, 제거된 컬럼 수 = 30
  반복 40: Max VIF = 8727.60, 제거된 컬럼 수 = 40
  반복 50: Max VIF = 3394.02, 제거된 컬럼 수 = 50
  반복 60: Max VIF = 1274.52, 제거된 컬럼 수 = 60
  반복 70: Max VIF = 706.65, 제거된 컬럼 수 = 70
  반복 80: Max VIF = 451.75, 제거된 컬럼 수 = 80
  반복 90: Max VIF = 306.02, 제거된 컬럼 수 = 90
  반복 100: Max VIF = 202.06, 제거된 컬럼 수 = 100

✓ VIF 제거 완료
  - 남은 컬럼 수: 250
  - 제거된 컬럼 수: 100
  - 제거된 컬럼 예시 (최대 5개): ['sensor_122', 'sensor_132', 'sensor_051', 'sensor_058', 'sensor_071']


##  IQR 기반 노이즈 드롭

**로직**:
1. 지정된 컬럼에 대해 Q1 - 1.5×IQR, Q3 + 1.5×IQR 범위 밖 값을 NaN으로 마킹
2. 해당 컬럼의 결측률이 ≥ 0.50이면 컬럼 자체를 제거

=> 센서 단위 품질 관리 QC에 해당되므로, 센서가 아예 불안정해서 데이터의 절반이 엉망인 경우 센서 자체가 문제이므로 센서를 없애는 것

① Stage A에서 이미 이상치 완화(Z-score) 를 했기 때문

② SECOM 데이터 자체가 결측 비율 낮음

Stage A에서 40% 이상 결측 드랍했으니까 남은 컬럼들엔 결측이 거의 없고, IQR로 NaN 마킹해도 0.5 비율 넘는 컬럼이 안 나옴.


**기본값**: 빈 리스트 (미적용)

In [9]:
# (옵션) IQR 기반 노이즈 드롭
NOISE_COLS = []  # 기본: 비어있음 (미적용)
IQR_MISSING_THRESHOLD = 0.50

dropped_iqr_cols = []

if len(NOISE_COLS) > 0:
    print(f"IQR 노이즈 드롭 적용 대상: {len(NOISE_COLS)}개 컬럼")
    
    for col in NOISE_COLS:
        if col not in X.columns:
            continue
        
        Q1 = X[col].quantile(0.25)
        Q3 = X[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # 범위 밖 값을 NaN으로 마킹
        X.loc[(X[col] < lower_bound) | (X[col] > upper_bound), col] = np.nan
        
        # 결측률 체크
        missing_ratio = X[col].isnull().mean()
        if missing_ratio >= IQR_MISSING_THRESHOLD:
            dropped_iqr_cols.append(col)
    
    # 고결측 컬럼 제거
    if len(dropped_iqr_cols) > 0:
        X = X.drop(columns=dropped_iqr_cols)
        print(f"  - IQR 후 고결측 컬럼 제거: {len(dropped_iqr_cols)}개")
    
    # 남은 NaN을 median으로 재대치
    for col in X.columns:
        if X[col].isnull().any():
            X[col].fillna(X[col].median(), inplace=True)
    
    print(f"✓ IQR 노이즈 드롭 완료")
else:
    print("✓ IQR 노이즈 드롭 스킵 (노이즈 컬럼 목록 비어있음)")

print(f"  - 제거된 컬럼 수: {len(dropped_iqr_cols)}")

✓ IQR 노이즈 드롭 스킵 (노이즈 컬럼 목록 비어있음)
  - 제거된 컬럼 수: 0


## 저장 & 로그 기록

**저장 파일**:
- `base_master_median.parquet`: 최종 전처리 데이터셋 (X + y)
- `preprocess_log_median.json`: 전처리 로그 (드롭/대치 내역)
- `preview_master_median.csv`: 상위 200행 미리보기

**로그 포함 항목**:
- 각 단계별 제거된 컬럼 목록
- 이상치 대치 통계
- 최종 shape 및 positive ratio

In [10]:
# 저장 & 로그 기록
# 최종 데이터셋 결합
df_final = X.copy()
df_final['label'] = y.values

# Parquet 저장
output_parquet = DIR_PROC / 'base_master_mean.parquet'
df_final.to_parquet(output_parquet, index=False)
print(f"✓ 최종 데이터셋 저장: {output_parquet}")
print(f"  - Shape: {df_final.shape}")

# 로그 생성
log_data = {
    'version': 'mean',
    'dropped_missing_cols': cols_to_drop_missing,
    'dropped_lowvar': cols_to_drop_lowvar,
    'dropped_vif': dropped_vif_cols,
    'dropped_iqr': dropped_iqr_cols,
    'outlier_replaced_counts': outlier_replaced_counts,
    'final_shape': {
        'rows': int(df_final.shape[0]),
        'cols': int(df_final.shape[1])
    },
    'positive_ratio': float(y.mean()),
    'preprocessing_steps': [
        'label_encoding',
        'missing_ratio_drop',
        'man_imputation',
        'low_variance_drop',
        'vif_multicollinearity',
        'zscore_outlier_mitigation',
        'iqr_noise_drop'
    ]
}

# 로그 저장
output_log = DIR_PROC / 'preprocess_log_mean.json'
with open(output_log, 'w', encoding='utf-8') as f:
    json.dump(log_data, f, indent=2, ensure_ascii=False)
print(f"✓ 전처리 로그 저장: {output_log}")

# 미리보기 저장 (상위 200행)
output_preview = DIR_INTERIM / 'preview_master_mean.csv'
df_final.head(200).to_csv(output_preview, index=False)
print(f"✓ 미리보기 저장: {output_preview}")

# 요약 출력
print("\n" + "="*60)
print("전처리 완료 요약")
print("="*60)
print(f"최종 컬럼 수: {df_final.shape[1] - 1} (+ label)")
print(f"최종 샘플 수: {df_final.shape[0]}")
print(f"Positive ratio: {y.mean():.4f}")
print(f"\n제거된 컬럼 총계:")
print(f"  - 결측률: {len(cols_to_drop_missing)}")
print(f"  - 저분산: {len(cols_to_drop_lowvar)}")
print(f"  - VIF: {len(dropped_vif_cols)}")
print(f"  - IQR: {len(dropped_iqr_cols)}")
print("="*60)

✓ 최종 데이터셋 저장: /Users/mealkuo/Desktop/capstone02_project/data/processed/base_master_mean.parquet
  - Shape: (1567, 251)
✓ 전처리 로그 저장: /Users/mealkuo/Desktop/capstone02_project/data/processed/preprocess_log_mean.json
✓ 미리보기 저장: /Users/mealkuo/Desktop/capstone02_project/data/interim/preview_master_mean.csv

전처리 완료 요약
최종 컬럼 수: 250 (+ label)
최종 샘플 수: 1567
Positive ratio: 0.0664

제거된 컬럼 총계:
  - 결측률: 32
  - 저분산: 116
  - VIF: 100
  - IQR: 0


## 빠른 검증 체크

데이터 품질을 최종 확인합니다:
- NaN 잔여 여부
- 컬럼 수 변화
- 라벨 분포
- 기본 통계량

In [11]:
# 빠른 검증 체크
print("[검증 체크]")
print("="*60)

# 1. NaN 체크
nan_sum = df_final.isnull().sum().sum()
print(f"1. NaN 총 개수: {nan_sum}")
assert nan_sum == 0, "⚠️  NaN이 남아있습니다!"
print("   ✓ NaN 없음 확인")

# 2. 컬럼 수 확인
print(f"\n2. 컬럼 수 (라벨 제외): {df_final.shape[1] - 1}")
print(f"   - 원본 컬럼 수와 비교하여 감소 확인")

# 3. 라벨 분포
print(f"\n3. 라벨 분포:")
print(df_final['label'].value_counts().sort_index())
print(f"   - Positive ratio: {df_final['label'].mean():.4f}")

# 4. 기본 통계량 (일부)
print(f"\n4. 기본 통계량 (첫 5개 피처):")
print(df_final.iloc[:, :5].describe().loc[['mean', 'std', 'min', 'max']])

print("\n" + "="*60)
print("✓ 모든 검증 통과")
print("="*60)

[검증 체크]
1. NaN 총 개수: 0
   ✓ NaN 없음 확인

2. 컬럼 수 (라벨 제외): 250
   - 원본 컬럼 수와 비교하여 감소 확인

3. 라벨 분포:
label
0    1463
1     104
Name: count, dtype: int64
   - Positive ratio: 0.0664

4. 기본 통계량 (첫 5개 피처):
      sensor_081  sensor_486  sensor_171  sensor_239  sensor_291
mean   -0.018531  185.823012    0.684330    0.004723    0.098236
std     0.048847  192.898602    0.157418    0.001448    0.056613
min    -0.143700    0.000000    0.297900    0.001300    0.041600
max     0.118600  850.602400    1.153000    0.009800    0.552800

✓ 모든 검증 통과


### 추가 수정

② 상관 필터링 드랍 리스트를 로그에 추가

③ 로그 & 파일 다시 저장



In [12]:
log_data = {
    'version': 'mean',
    'dropped_missing_cols': cols_to_drop_missing,
    'dropped_lowvar': cols_to_drop_lowvar,
    'dropped_corr': drop_corr,  # ✅ 추가
    'dropped_vif': dropped_vif_cols,
    'dropped_iqr': dropped_iqr_cols,
    'outlier_replaced_counts': outlier_replaced_counts,
    'final_shape': {
        'rows': int(df_final.shape[0]),
        'cols': int(df_final.shape[1])
    },
    'positive_ratio': float(df_final['label'].mean()),
    'preprocessing_steps': [
        'label_encoding',
        'missing_ratio_drop',
        'mean_imputation',
        'low_variance_drop',
        'correlation_filter',   # ✅ 추가
        'vif_multicollinearity',
        'zscore_outlier_mitigation',
        'iqr_noise_drop'
    ]
}

# 덮어 쓰기 저장
output_log = DIR_PROC / 'preprocess_log_mean.json'
with open(output_log, 'w', encoding='utf-8') as f:
    json.dump(log_data, f, indent=2, ensure_ascii=False)

print("✓ 로그 갱신 완료")

✓ 로그 갱신 완료
